In [3]:
import utils
from typing import List, Dict
from pathlib import Path

test_questions: Dict[str, Dict] = utils.load_json(Path("qasper/test_questions.json"))
for idx, (question_id, question_data) in enumerate(test_questions.items()):
        print(f"Processing `{question_data['question']}`...")

Processing `How big is the ANTISCAM dataset? `...
Processing `How is intent annotated?`...
Processing `What are the baselines outperformed by this work?`...
Processing `What are the evaluation metrics and criteria used to evaluate the model performance?`...
Processing `What is the accuracy of this model compared to sota?`...
Processing `What previous methods do they compare against?`...
Processing `What is their evaluation metric?`...
Processing `Are their methods fully supervised?`...
Processing `Do they build a dataset of rumors?`...
Processing `What languages do they evaluate their methods on?`...
Processing `How do they define rumors?`...
Processing `What baselines did they compare with?`...
Processing `Which tasks are explored in this paper?`...
Processing `Which NER dataset do they use?`...
Processing `How do they incorporate direction and relative distance in attention?`...
Processing `Do they outperform current NER state-of-the-art models?`...
Processing `What was their accurac

In [ ]:
import json
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Union
from sentence_transformers import SentenceTransformer, util
import jsonlines
import torch
import re
from transformers import T5ForConditionalGeneration, T5Tokenizer
import time
import os
import argparse

os.environ['HF_HOME'] = '/workspace/P76125041/.cache/'

# CUDA_DEVICE = 1

RETRIEVER = "sbert"
READER = "unified"
MODE = "full"
TOPK = 3

retriever_map: Dict[str, str] = {
    "sbert": "sentence-transformers/multi-qa-mpnet-base-cos-v1",
    "stella": "/workspace/P76125041/models/stella_en_400M_v5"
}

reader_map: Dict[str, str] = {
    "unified": "allenai/unifiedqa-v2-t5-3b-1363200",
}

def format_paragraphs(raw_paras: List[dict], mode: str) -> List[str]:
    if mode == "full":
        return [
            f"Section Title: {para.get('section_name', '[UNNAMED SECTION]')}\n\nText: {para.get('text', '')}"
            for para in raw_paras
        ]
    elif mode == "toptitle":
        return [
            f"Section Title: {re.sub(r':::.*', '', para.get('section_name', '[UNNAMED SECTION]')).strip()}\n\nText: {para.get('text', '')}"
            for para in raw_paras
        ]
    else:
        return [para.get('text', '') for para in raw_paras]



In [ ]:
# load all paper contents from test set
data_file: Path = Path("qasper/test_papers.json")
with open(data_file, "r") as file:
    test_papers: Dict[str, Dict] = json.load(file)

# load all questions from test set
questions_file: Path = Path("qasper/test_questions.json")
with open(questions_file, "r") as file:
    test_questions: Dict[str, Dict] = json.load(file)

# initialize retriever
### SBERT
embedding_model = SentenceTransformer(retriever_map.get(RETRIEVER))

# prepare paragraph embeddings for each paper [approx time: 5min]
all_paper_ids: List[str] = [q["from_paper"] for q in test_questions.values()]
paper_para_embeddings: Dict[str, List[List[float]]] = {}
for paper_id in all_paper_ids:
    paragraphs: List[str] = format_paragraphs(test_papers[paper_id].values(), MODE)
    para_embeddings: List[List[float]] = embedding_model.encode(paragraphs)
    paper_para_embeddings[paper_id] = para_embeddings

# Save paragraph embeddings to JSON
embedding_file = f"qasper/test_paper_embeddings_{MODE}.json"
with open(embedding_file, "w+", encoding="utf-8") as f:
    json.dump(paper_para_embeddings, f, indent=4)


In [3]:
import json
from pathlib import Path
# from sentence_transformers import SentenceTransformer
import re
import time
import os
import argparse
from typing import List, Dict, Set

# load all questions from test set
questions_file: Path = Path("qasper/test_questions.json")
with open(questions_file, "r") as file:
    test_questions: Dict[str, Dict] = json.load(file)

all_paper_ids: Set[str] = set()
paper_ids: List[str] = []
for question in test_questions.values():
    all_paper_ids.add(question["from_paper"])
    paper_ids.append(question["from_paper"])

print(len(all_paper_ids))
print(len(paper_ids))


416
1451


In [8]:
all_paper_ids = list(test_questions.values())

In [11]:
from collections import Counter
from typing import List
# load all questions from test set
questions_file: Path = Path("qasper/test_questions.json")

with open(questions_file, "r") as file:
    test_questions: Dict[str, Dict] = json.load(file)

def find_duplicates(lst: List[int]) -> List[int]:
    """
    Finds and returns duplicate elements in the list.
    """
    return [item for item, count in Counter(lst).items() if count > 1]

# Example usage:
lst = [1, 2, 3, 4, 5, 2, 3, 6]
duplicates = find_duplicates(test_questions.keys())
print(duplicates)  # Output: [2, 3]
len(test_questions)


[]


1451

In [ ]:
paper_id = "1911.10742"
MODE = "full"
embedding_file = f"qasper/test_paper_embeddings_{MODE}.json"

# load embeddings
with open(embedding_file, "r") as file:
    paper_para_embeddings: Dict[str, List[List[float]]] = json.load(file)

print(len(paper_para_embeddings[paper_id]))
print(paper_para_embeddings[paper_id][:20])

In [ ]:
from pathlib import Path
from typing import List, Dict
import json

full_title_error_file: Path = Path("results/full_titles_top3/error_cases.json")
no_title_error_file: Path = Path("results/no_titles/error_cases.json")

# get all paper contents from test set
with open(full_title_error_file, "r") as file:
    full_title_errors: Dict[str, Dict] = json.load(file)
    
# get all questions from test set
with open(no_title_error_file, "r") as file:
    no_title_errors: Dict[str, Dict] = json.load(file)

print(full_title_errors.values())

In [1]:
import os
print(os.environ.get('CUDA_HOME'))
total time (sec): 725.7992417812347

None


In [1]:
hi = ["1", "2", "3"]
print(str(hi))

['1', '2', '3']


In [2]:
from sentence_transformers import SentenceTransformer

model_path = "3"

model = SentenceTransformer(
        "dunzhang/stella_en_400M_v5", trust_remote_code=True, device="cuda:1"
    ).cuda(1)

model.save(model_path)

/workspace/P76125041/miniconda3/envs/exp-b4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/P76125041/miniconda3/envs/exp-b4/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/workspace/P76125041/miniconda3/envs/exp-b4/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Some w

In [ ]:
question_text = "Which three variants of cross-validation are examined?"



In [ ]:
from sentence_transformers import SentenceTransformer


query_prompt_name = "s2p_query"
model = SentenceTransformer(
"dunzhang/stella_en_400M_v5", trust_remote_code=True, device="cuda"
).cuda()

query_embedding = model.encode([question_text], prompt_name=query_prompt_name)
para_embeddings = model.encode(paragraphs)
# print(query_embedding.shape, para_embeddings.shape)
# (2, 1024) (2, 1024)

similarities = model.similarity(query_embedding, para_embeddings)